### Reconcile Bank data with Beals data

- Check Beals cumulative payments
    - puts credits/debits into a single column and keeps track of cumulative values - a records with an 'Amount payable' 
      with a non-zero value shows records have been carried forward from previous year or are to be carried forward to
      next.
- Reconcile Bank payments against Beals
    - iterate bank data for each payment from Beals, checking for corresponding data in Beals transactions list.
    - check for Bank records not reconciled to Beals payment
    - check for Beals records with not corresponding Bank payment
    - find Beals payments that include a transaction from previous year (identified by non zero cumulative amount)
    - find Beals transactions that have not been paid for and so are carried forward to next year (reconcile=0 flag)
- Create output spreadsheets
    - included is combo ledger of bank payments received along with Beals transactions in order to check balances

In [1]:
import numpy as np
import pandas as pd

In [2]:
import time
import os.path
import shutil

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

In [3]:
#path = 'M:/My Documents/Business/Bugisiw Ltd/TaxReturn/TaxReturn_2020-2021/LLP accounts/'
taxreturn_yr='TaxReturn_2022-2023'
path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\'+taxreturn_yr+'\\LLP accounts\\'
startdatestr='2022-04-06'
enddatestr='2022-07-31'

In [4]:
# Load bank files
#df6=pd.read_csv(path+'6045_s1.csv', parse_dates=['Date'], dayfirst=True)
#df3=pd.read_csv(path+'3072_s1.csv', parse_dates=['Date'], dayfirst=True)
input_file_6045 = path + '6045_categorised.xlsx'
df6=pd.read_excel(input_file_6045,sheet_name='6045')
df6=df6[~df6.Account.isnull()]
input_file_3072 = path + '3072_categorised.xlsx'
df3=pd.read_excel(input_file_3072,sheet_name='3072')
df3=df3[~df3.Account.isnull()]

In [5]:
# Load Beals transactions
#dfB=pd.read_csv(path+'beals.csv', parse_dates=['Date Created'], dayfirst=True)
dfB=pd.read_excel(path+'beals.xlsx', sheet_name='beals', parse_dates=['Date Created'], dayfirst=True)
dfB.insert(0,'Date', dfB['Date Created'].dt.date)
dfB=dfB.sort_values(by=['ID','Date Created'])
dfB['Date'] =  pd.to_datetime(dfB.Date)
#dfB['Item Description'] = dfB.ID + ': ' + dfB['Item Description']   

In [6]:
startdate = pd.to_datetime(startdatestr).date()
enddate = pd.to_datetime(enddatestr).date()
dfB=dfB[(dfB.Date >= startdate)&(dfB.Date <= enddate)]
df6=df6[(df6.Date >= startdate)&(df6.Date <= enddate)]
df3=df3[(df3.Date >= startdate)&(df3.Date <= enddate)]

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.
C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Comparing Series of datetimes w

### Add cumulative payments to Beals transactions
- it's easier to see missing transacations when there is a cumulative figure

In [7]:
# Create amount column
for index, row in dfB.iterrows():
    if(row['Credit Amount']>0):
        dfB.at[index,'Amount']=-round(row['Credit Amount'],2)
    elif(row['Debit Amount']>0):
        dfB.at[index,'Amount']=round(row['Debit Amount'],2)

In [8]:
# Track cumulative amounts between payments
cumamount=0.0;
lastID=''
for index, row in dfB.iterrows():
    if(row.ID != lastID):
        cumamount=row.Amount
    else:
        cumamount=cumamount + row.Amount

    dfB.at[index,'CumAmount'] = round(cumamount,2)
    if('Amount payable' in row['Item Description']):
        cumamount=0.0
    lastID=row.ID

### Reconcile Bank payments against Beals

Iterate bank data for each payment from Beals, checking for corresponding data in Beals transactions list.

Mark reconciled data in bank and reconciled Beals payments with 1, mark Beals transactions with account they were paid into.

In [9]:
from datetime import datetime, timedelta

def find_last_rent_received(payment_index, property_code, df_beals):
    rec=df_beals[(df_beals.ID==property_code)&(df_beals['Item Description'].str.contains('Rent for period'))&(df_beals.index<payment_index)]
    if not rec.empty:
        return rec['Item Description'].iloc[-1]
    else:
        return 'NOT FOUND'

def reconcile_bank_with_beals(df_bank, df_beals,bank_label):
    lookback = 6
    errorcount = 0
    for index, row in df_bank[df_bank['Memo'].str.contains("BEALS ESTATE AGENT", na=False)].iterrows():
        # Lookback 5 days for payment to appear in Beals statement
        print('Reconciling ' + str(row.Date) + ' ' + row.Memo + ' ' + str(row.Amount))

        startdate = row.Date - timedelta(days=lookback)
        enddate = row.Date
        rec=df_beals.loc[(df_beals['ID']==row.Property)&(df_beals['Debit Amount']==row.Amount)&(df_beals['Date']>=startdate)&(df_beals['Date']<=enddate)&(df_beals.ReconciledBank!=1)]
        if not rec.empty:
            print( str(rec.iloc[0].name)+' With '+ str(rec.Date.iloc[0]) + ' ' + rec['Item Description'].iloc[0] + ' ' + str(rec['Debit Amount'].iloc[0]))
            df_bank.at[index, 'ReconciledBeals']=1
            df_bank.at[index, 'Description']=find_last_rent_received(rec.iloc[0].name,row.Property,df_beals)
            df_beals.at[rec.iloc[0].name, 'ReconciledBank']=1
            df_beals.at[rec.iloc[0].name, 'ReconciledBankAccount']=bank_label
        else:
            print('CANNOT RECONCILE')
            errorcount = errorcount + 1

    print('Errors=' +  str(errorcount))

In [10]:
df6.loc[df6['Memo'].str.contains("BEALS ESTATE AGENT", na=False),'ReconciledBeals']=0
df3.loc[df3['Memo'].str.contains("BEALS ESTATE AGENT", na=False),'ReconciledBeals']=0
dfB['ReconciledBank']=np.nan
dfB['ReconciledBankAccount']=np.nan

#### Reconcile 6045 with Beals

In [11]:
reconcile_bank_with_beals(df6,dfB,6045)

Reconciling 2022-04-07 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 470.32
CANNOT RECONCILE
Reconciling 2022-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 1412-14 ALHAM BGC 452.0
12 With 2022-04-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 14, 12-14 Alhambra Road, Southsea 452.0
Reconciling 2022-04-11 00:00:00 BEALS ESTATE AGENT    FLAT 2171 FAWCETT  BGC 416.76
34 With 2022-04-07 00:00:00 Amount payable to Mr Mathew Tucker - Flat 2, 171 Fawcett Road, Southsea 416.76
Reconciling 2022-04-11 00:00:00 BEALS ESTATE AGENT    FLAT 3163 FRATTON  BGC 151.74
56 With 2022-04-07 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 163 Fratton Road, Portsmouth 151.74
Reconciling 2022-04-11 00:00:00 BEALS ESTATE AGENT    FLAT 1012-14 ALHAM BGC 205.7
5 With 2022-04-07 00:00:00 Amount payable to Mr Mathew Tucker - Flat 10, 12-14 Alhambra Road, Southsea 205.7
Reconciling 2022-04-11 00:00:00 BEALS ESTATE AGENT    FLAT 1169 FAWCETT  BGC 420.96
40 With 2022-04-07 00:00:00 Amount payable to 

Reconciling 2022-06-30 00:00:00 BEALS ESTATE AGENT    	FLAT 1012-14 ALHAM BG 311.45
222 With 2022-06-28 00:00:00 Amount payable to Mr Mathew Tucker - Flat 10, 12-14 Alhambra Road, Southsea 311.45
Reconciling 2022-07-06 00:00:00 BEALS ESTATE AGENT    	FLAT 2171 FAWCETT BGC 175.36
372 With 2022-07-04 00:00:00 Amount payable to Mr Mathew Tucker - Flat 2, 171 Fawcett Road, Southsea 175.36
Reconciling 2022-07-07 00:00:00 BEALS ESTATE AGENT    	196A KINGSTON ROAD BG 470.32
389 With 2022-07-05 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
Reconciling 2022-07-08 00:00:00 BEALS ESTATE AGENT    	FLAT 1412-14 ALHAM BG 452.0
353 With 2022-07-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 14, 12-14 Alhambra Road, Southsea 452.0
Reconciling 2022-07-12 00:00:00 BEALS ESTATE AGENT    	FLAT 68 ALHAMBRA R BG 416.76
395 With 2022-07-08 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 8 Alhambra Road, Southsea 416.76
Reconciling 2022-07-13 00:00:00 BEALS E

#### Reconcile 3072 with Beals

In [12]:
reconcile_bank_with_beals(df3,dfB,3072)

Reconciling 2022-04-07 00:00:00 BEALS ESTATE AGENT    FLAT 78 ALHAMBRA R BGC 452.0
CANNOT RECONCILE
Reconciling 2022-04-11 00:00:00 BEALS ESTATE AGENT    FLAT 1216-18 ALHAM BGC 136.27
93 With 2022-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 136.27
Reconciling 2022-04-11 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 200.06
80 With 2022-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 200.06
Reconciling 2022-04-11 00:00:00 BEALS ESTATE AGENT    FLAT 1612-14 ALHAM BGC 422.29
89 With 2022-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 16, 12-14 Alhambra Road, Southsea 422.29
Reconciling 2022-04-22 00:00:00 BEALS ESTATE AGENT    FLAT 1216-18 ALHAM BGC 284.74
96 With 2022-04-20 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 284.74
Reconciling 2022-04-27 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 208.26
83 With 2022-04-23 00:00:00 A

#### Tidy reconciliation flag

In [13]:
# Backfill reconciliation flag so we know what bank account all Beals transactions relate to
dfB['ReconciledBank'] = dfB.groupby('ID')['ReconciledBank'].bfill()
dfB['ReconciledBank'].fillna(value=0, inplace=True)
dfB['ReconciledBankAccount'] = dfB.groupby('ID')['ReconciledBankAccount'].bfill()
dfB['ReconciledBankAccount'] = dfB.groupby('ID')['ReconciledBankAccount'].ffill()

#### Error check - Bank transactions not reconciled to Beals Payments

- check Beals data is complete if there are bank payments without a Beals record

In [14]:
df6.loc[(df6['Memo'].str.contains("BEALS ESTATE AGENT"))&(df6.ReconciledBeals==0)]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category,ReconciledBeals
6,2022-04-07,20-74-09 60458872,470.32,DIRECTDEP,BEALS ESTATE AGENT 196A KINGSTON ROAD BGC,196AKIN,NaN,BealsRentalIncome,0.0


In [15]:
df3.loc[(df3['Memo'].str.contains("BEALS ESTATE AGENT"))&(df3.ReconciledBeals==0)]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category,ReconciledBeals
1,2022-04-07,20-53-97 30728691,452.0,DIRECTDEP,BEALS ESTATE AGENT FLAT 78 ALHAMBRA R BGC,F78ALH,NaN,BealsRentalIncome,0.0


#### Error check - Beals Payments not reconciled to Bank transaction - BEALS SAID THEY PAID BUT NOTHING RECEIVED
- there could be Beals payments at end of year which were paid into bank after year end
- need to contact Beals if there payments on other dates 

In [16]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB['Item Description'].str.contains("Tucker"))&(dfB.ReconciledBank==0)]

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
399,2022-07-28,F68ALH,903833,2022-07-28 11:32:00,"Amount payable to Mr Mathew Tucker - Flat 6, 8...",Statement,271665,47.75,0.0,PM,47.75,0.0,0.0,6045.0


In [17]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB['Item Description'].str.contains("Valentino"))&(dfB.ReconciledBank==0)]

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount


#### Payments for Beals transactions from previous year

This section works on the idea that when an amount is payed from Beals ("Amount payable"), this payment should clear the cumulative amount back to 0. If it is not 0, then there is something that might need looking at.

The dates on below transactions should all be around start of period. The cumulative amounts of £6 indicate Legislation & Compliance charges in previous period but paid for in this period. Dates later in year can indicate missing data in Beals spreadsheet so check the Beals paper statements match the Beals data.

In [18]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
12,2022-04-06,F141214ALH,879859,2022-04-06 12:43:00,"Amount payable to Mr Mathew Tucker - Flat 14, ...",Statement,261097,452.00,0.0,PM,452.00,-6.00,1.0,6045.0
5,2022-04-07,F101214ALH,880097,2022-04-07 13:08:00,"Amount payable to Mr Mathew Tucker - Flat 10, ...",Statement,261177,205.70,0.0,PM,205.70,-6.00,1.0,6045.0
56,2022-04-07,F3163FRA,880097,2022-04-07 13:08:00,"Amount payable to Mr Mathew Tucker - Flat 3, 1...",Statement,261182,151.74,0.0,PM,151.74,-6.00,1.0,6045.0
34,2022-04-07,F2171FAW,880097,2022-04-07 13:08:00,"Amount payable to Mr Mathew Tucker - Flat 2, 1...",Statement,261181,416.76,0.0,PM,416.76,-6.00,1.0,6045.0
63,2022-04-07,F71214ALH,880097,2022-04-07 13:08:00,"Amount payable to Mr Mathew Tucker - Flat 7, 1...",Statement,261185,206.15,0.0,PM,206.15,-6.00,1.0,6045.0
40,2022-04-07,F1169FAW,880097,2022-04-07 13:08:00,"Amount payable to Mr Mathew Tucker - Flat 1, 1...",Statement,261176,420.96,0.0,PM,420.96,-6.00,1.0,6045.0
52,2022-04-08,F68ALH,880236,2022-04-08 10:35:00,"Amount payable to Mr Mathew Tucker - Flat 6, 8...",Statement,261233,452.00,0.0,PM,452.00,-6.00,1.0,6045.0
72,2022-04-11,F1171FAW,880467,2022-04-11 12:34:00,"Amount payable to Mr Mathew Tucker - Flat 1, 1...",Statement,261295,635.20,0.0,PM,635.20,-6.00,1.0,6045.0
19,2022-04-12,F61618ALH,880985,2022-04-12 12:13:00,"Amount payable to Mr Mathew Tucker - Flat 6, 1...",Statement,261680,416.75,0.0,PM,416.75,-6.00,1.0,6045.0
44,2022-04-12,SHOP196KIN,880985,2022-04-12 12:13:00,"Amount payable to Mr Mathew Tucker - Shop, 196...",Statement,261684,314.60,0.0,PM,314.60,-6.00,1.0,6045.0


**** It looks like Flat 10 12-14 went wrong somewhere ****
- I checked this and the problem is things are a bit in the wrong order in Beals transactions files - it balances out to 0 in the end

In [19]:
dfB['Amount'].loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sum()

5725.38

In [20]:
# Output to file
#dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date']).to_excel(path+'6045_prevyr.xlsx',sheet_name='6045', index=False)

In [21]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
93,2022-04-07,F121618ALH,880097,2022-04-07 13:08:00,Amount payable to Ms Ivana Valentino - Flat 12...,Statement,261178,136.27,0.0,PM,136.27,-6.0,1.0,3072.0
89,2022-04-07,F161214ALH,880097,2022-04-07 13:08:00,Amount payable to Ms Ivana Valentino - Flat 16...,Statement,261179,422.29,0.0,PM,422.29,-6.0,1.0,3072.0
80,2022-04-07,F51214ALH,880097,2022-04-07 13:08:00,"Amount payable to Ms Ivana Valentino - Flat 5,...",Statement,261184,200.06,0.0,PM,200.06,-6.0,1.0,3072.0


In [22]:
dfB['Amount'].loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sum()

758.6200000000001

In [23]:
# Output to file
#dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date']).to_excel(path+'3072_prevyr.xlsx',sheet_name='3072', index=False)

#### Payments carried forward to next year
- Often the records in the middle of the year are a result of Beals charging Legal & Compliance fees for flats which we took over after the tenant moved out

In [24]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['ID','Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
390,2022-07-28,196AKIN,903686,2022-07-28 00:32:00,Legislation & Compliance - 196a Kingston Road...,Invoice,360779,6.00,0.00,EX,6.00,6.00,0.0,6045.0
350,2022-07-28,F101214ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 10, 12-14 Alh...",Invoice,360703,6.00,0.00,EX,6.00,6.00,0.0,6045.0
365,2022-07-28,F111618ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 11, 16-18 Alh...",Invoice,360746,6.00,0.00,EX,6.00,6.00,0.0,6045.0
382,2022-07-28,F1169FAW,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 1, 169 Fawcet...",Invoice,360761,6.00,0.00,EX,6.00,6.00,0.0,6045.0
419,2022-07-28,F1171FAW,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 1, 171 Fawcet...",Invoice,360763,6.00,0.00,EX,6.00,6.00,0.0,6045.0
369,2022-07-28,F1321LON,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 1, 321 London...",Invoice,360811,12.00,0.00,EX,12.00,12.00,0.0,6045.0
354,2022-07-28,F141214ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 14, 12-14 Alh...",Invoice,360708,6.00,0.00,EX,6.00,6.00,0.0,6045.0
376,2022-07-28,F2171FAW,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 2, 171 Fawcet...",Invoice,360765,6.00,0.00,EX,6.00,6.00,0.0,6045.0
406,2022-07-28,F3163FRA,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 3, 163 Fratto...",Invoice,360755,6.00,0.00,EX,6.00,6.00,0.0,6045.0
361,2022-07-28,F61618ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 6, 16-18 Alha...",Invoice,360753,6.00,0.00,EX,6.00,6.00,0.0,6045.0


In [25]:
dfB['Amount'].loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sum()

78.0

In [26]:
# Output carried forward to file
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date']).to_excel(path+'6045_carriedfwd.xlsx',sheet_name='6045', index=False)

In [27]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['ID','Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
426,2022-07-28,F121618ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 12, 16-18 Alh...",Invoice,360747,6.0,0.0,EX,6.0,6.0,0.0,3072.0
432,2022-07-28,F161214ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 16, 12-14 Alh...",Invoice,360711,6.0,0.0,EX,6.0,6.0,0.0,3072.0
439,2022-07-28,F51214ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 5, 12-14 Alha...",Invoice,360714,6.0,0.0,EX,6.0,6.0,0.0,3072.0
443,2022-07-28,F78ALH,903686,2022-07-28 00:32:00,"Legislation & Compliance - Flat 7, 8 Alhambra...",Invoice,360856,6.0,0.0,EX,6.0,6.0,0.0,3072.0


In [28]:
dfB['Amount'].loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sum()

24.0

In [29]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date']).to_excel(path+'3072_carriedfwd.xlsx',sheet_name='3072', index=False)

### Output files
- save Bank files with Beals payment reconciliation flag
- save Beals files with bank reconciliation flag, bank account it was paid into and cumulative amount

In [30]:
file6=backup_file(path+'6045_categorisedAndBealsReconciled.xlsx')
file3=backup_file(path+'3072_categorisedAndBealsReconciled.xlsx')
fileB=backup_file(path+'beals_bankReconciled.xlsx')
df6.to_excel(file6,sheet_name='6045', index=False)
df3.to_excel(file3,sheet_name='3072', index=False)
dfB.to_excel(fileB,sheet_name='beals', index=False)

### Create Beals/Bank reconciliation check
- this file can be used to check that the bank deposits minus agency fees match the Beals payments

In [31]:
dfB.rename(columns={'ReconciledBankAccount':'BankAccount'},inplace=True)
dfB.rename(columns={'Item Description':'PostNarrative'},inplace=True)
dfB.rename(columns={'ID':'Property'},inplace=True)

In [32]:
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Account']='0001'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Account Description']='Rental Income'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Category']='RentalIncome'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Account']='0002'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Account Description']='Rental Income'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Category']='RentalIncome'

dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Category']='AgentFees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Category']='AgentFees'

dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Category']='AgentFees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Category']='AgentFees'

dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Category']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Category']='Payment'

dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Category']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Category']='Payment'

dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account']='3140'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account Description']='Repairs, renewals, maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Category']='Maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account']='3140'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account Description']='Repairs, renewals, maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Category']='Maintenance'


In [33]:
# Check nothing in Beals remains uncategorised
dfB[dfB.Category.isnull()]

,Date,Property,Activity I D,Date Created,PostNarrative,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Amount,CumAmount,ReconciledBank,BankAccount,Account,Account Description,Category
98,2022-04-28,F161618ALH,884442,2022-04-28 00:32:00,"Legislation & Compliance - Flat 16, 16-18 Alha...",Invoice,349086,6.0,0.0,EX,6.0,6.0,0.0,NaN,NaN,NaN,NaN
209,2022-05-28,F161618ALH,890834,2022-05-28 00:32:00,"Legislation & Compliance - Flat 16, 16-18 Alha...",Invoice,353112,6.0,0.0,EX,6.0,12.0,0.0,NaN,NaN,NaN,NaN
343,2022-06-28,F161618ALH,896912,2022-06-28 00:32:00,"Legislation & Compliance - Flat 16, 16-18 Alha...",Invoice,356889,6.0,0.0,EX,6.0,18.0,0.0,NaN,NaN,NaN,NaN
2,2022-04-28,F1846ALH,884442,2022-04-28 00:32:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,348968,6.0,0.0,EX,6.0,6.0,0.0,NaN,NaN,NaN,NaN
101,2022-05-28,F1846ALH,890834,2022-05-28 00:32:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,352996,6.0,0.0,EX,6.0,12.0,0.0,NaN,NaN,NaN,NaN
212,2022-06-28,F1846ALH,896912,2022-06-28 00:32:00,"Legislation & Compliance - Flat 18, 4-6 Alham...",Invoice,356780,6.0,0.0,EX,6.0,18.0,0.0,NaN,NaN,NaN,NaN
1,2022-04-28,F1946ALH,884442,2022-04-28 00:32:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,348970,6.0,0.0,EX,6.0,6.0,0.0,NaN,NaN,NaN,NaN
100,2022-05-28,F1946ALH,890834,2022-05-28 00:32:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,352998,6.0,0.0,EX,6.0,12.0,0.0,NaN,NaN,NaN,NaN
211,2022-06-28,F1946ALH,896912,2022-06-28 00:32:00,"Legislation & Compliance - Flat 19, 4-6 Alham...",Invoice,356782,6.0,0.0,EX,6.0,18.0,0.0,NaN,NaN,NaN,NaN
0,2022-04-28,F2046ALH,884442,2022-04-28 00:32:00,"Legislation & Compliance - Flat 20, 4-6 Alham...",Invoice,348971,6.0,0.0,EX,6.0,6.0,0.0,NaN,NaN,NaN,NaN


In [34]:
cols=['BankAccount','Date','Property','PostNarrative','Amount','Account','Account Description','Category']

In [35]:
dfGr6=dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045)][cols]
dfGr3=dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072)][cols]
dfLc6=dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045)][cols]
dfLc3=dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072)][cols]
dfMg6=dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045)][cols]
dfMg3=dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072)][cols]
dfOth6=dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045)][cols]
dfOth3=dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072)][cols]

In [36]:
df6.rename(columns={'Account':'BankAccount'},inplace=True)
df6.rename(columns={'Memo':'PostNarrative'},inplace=True)
df6['BankAccount']=6045
df6['Account Description']='Payment'
df6.loc[(df6.Category=='BealsRentalIncome'),'Account']='4801'

df3.rename(columns={'Account':'BankAccount'},inplace=True)
df3.rename(columns={'Memo':'PostNarrative'},inplace=True)
df3['BankAccount']=3072
df3['Account Description']='Payment'
df3.loc[(df6.Category=='BealsRentalIncome'),'Account']='4801'

In [37]:
dfBk6=df6.loc[(df6.Category=='BealsRentalIncome')][cols]
dfBk3=df3.loc[(df3.Category=='BealsRentalIncome')][cols]

In [38]:
dfAll6=pd.concat([dfGr6,dfLc6,dfMg6,dfBk6,dfOth6]).sort_values(by=['Property','Date'])
dfAll6.reset_index(drop=True,inplace=True)
dfAll3=pd.concat([dfGr3,dfLc3,dfMg3,dfBk3,dfOth3]).sort_values(by=['Property','Date'])
dfAll3.reset_index(drop=True,inplace=True)

In [39]:
journals=path + 'journals\\'

beals_output_file=backup_file(journals + '3.6.AgentFeesRentalIncomeBankings_Beals_6045.xlsx')
writer = pd.ExcelWriter(beals_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
dfGr6.to_excel(writer,sheet_name='GrossRent', index=False)
dfLc6.to_excel(writer,sheet_name='Legal', index=False)
dfMg6.to_excel(writer,sheet_name='MgtFee', index=False)
dfOth6.to_excel(writer,sheet_name='Other', index=False)
dfBk6.to_excel(writer,sheet_name='Bank', index=False)
dfAll6.to_excel(writer,sheet_name='All', index=False)
writer.save()

beals_output_file=backup_file(journals + '3.6.AgentFeesRentalIncomeBankings_Beals_3072.xlsx')
writer = pd.ExcelWriter(beals_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
dfGr3.to_excel(writer,sheet_name='GrossRent', index=False)
dfLc3.to_excel(writer,sheet_name='Legal', index=False)
dfMg3.to_excel(writer,sheet_name='MgtFee', index=False)
dfOth3.to_excel(writer,sheet_name='Other', index=False)
dfBk3.to_excel(writer,sheet_name='Bank', index=False)
dfAll3.to_excel(writer,sheet_name='All', index=False)
writer.save()